# Desktop Maps [More documentation to come!]

By Kenneth Burchfiel

Released under the MIT License

This program will create various maps that can serve as desktop backgrounds.

## Downloading shapefiles

I visited [the Census's shapefile website](https://www.census.gov/cgi-bin/geo/shapefiles/index.php) in order to download state, county, and zip-level boundaries. I chose to download 2021 boundaries rather than later versions so that Connecticut county boundaries would still be available.

I chose to download the following shapefiles:

1. Urban areas
2. Core-based statistical areas
3. Congressional Districts (the most recent ones available were from the 116th US Congress)
4. The US Coastline
5. Counties
6. Primary roads
7. States
8. Zip Code Tabulation Areas (ZCTAs)

Once I had downloaded each of these datasets' corresponding zipfiles, I went ahead and extracted them, then placed them into a 'Datasets/Shapefiles' folder within my home folder. (Some of these datasets were well above the 100-megabyte limit for GitHub uploads, so I added only a sample into this folder.)

In [1]:
import pandas as pd
import geopandas as gpd
import folium
import time
from selenium import webdriver
import os
from PIL import Image # I ran conda install pillow
# in order to get PIL set up on my conda environment.

In [2]:
path_to_shapefiles = '../../../../../Datasets/Shapefiles/'

## Reading in a polygon that provides a rough outline of the contiguous US:

(I created this file within geojson.io.)

In [3]:
gdf_contig_us_bounds = gpd.read_file('contiguous_us_bounds.json')
gdf_contig_us_bounds['geometry'] = gdf_contig_us_bounds['geometry'].to_crs('EPSG:4269')
gdf_contig_us_bounds['merge_key'] = 1
gdf_contig_us_bounds.rename(
    columns = {'geometry':'contig_us_bounds'}, inplace = True)
gdf_contig_us_bounds

,contig_us_bounds,merge_key
0,"POLYGON ((-125.96355 47.7781, -127.04332 47.30...",1


Note: I based the to_crs() call above on the following warning message I had received in an earlier version of the code:

UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4269
Right CRS: EPSG:4326

  gdf_roads['within_contig_us'] = gdf_roads['geometry'].within(

In [4]:
gdf_roads = gpd.read_file(
    path_to_shapefiles
    +'tl_2021_us_primaryroads/tl_2021_us_primaryroads.shp')
gdf_roads['geometry'] = gdf_roads['geometry'].simplify(tolerance = 0.001).copy()
gdf_roads['merge_key'] = 1
gdf_roads.query("RTTYP == 'I'", inplace = True) # Limits routes shown to
# US interstate highways
gdf_roads = gdf_roads.merge(gdf_contig_us_bounds, 
                            on = 'merge_key', how = 'left')
gdf_roads.drop('merge_key', axis = 1, inplace = True)
gdf_roads.head()

,LINEARID,FULLNAME,RTTYP,MTFCC,geometry,contig_us_bounds
0,1106073066667,W I- 20,I,S1100,"LINESTRING (-98.10768 32.61152, -98.07695 32.6...","POLYGON ((-125.96355 47.7781, -127.04332 47.30..."
1,110766972487,W I- 20,I,S1100,"LINESTRING (-102.76769 31.63855, -102.80296 31...","POLYGON ((-125.96355 47.7781, -127.04332 47.30..."
2,1104259293529,W I- 20,I,S1100,"LINESTRING (-103.3894 31.42685, -103.35124 31....","POLYGON ((-125.96355 47.7781, -127.04332 47.30..."
3,110206001621,W I- 20,I,S1100,"LINESTRING (-98.97076 32.37467, -99.01205 32.3...","POLYGON ((-125.96355 47.7781, -127.04332 47.30..."
4,110451633229,I- 785,I,S1100,"LINESTRING (-79.68662 36.0495, -79.6844 36.056...","POLYGON ((-125.96355 47.7781, -127.04332 47.30..."


## Cropping this list of roads to include only those within
# the contiguous US:

(This step ended up not being necessary, but I'm leaving this code in because it could serve as a useful reference for future projects that *do* require filtering results to ontly include those within a given boundary.)

In [5]:
gdf_roads['within_contig_us'] = gdf_roads['geometry'].within(
    gdf_roads['contig_us_bounds'])
gdf_roads.query("within_contig_us == True", inplace = True)
gdf_roads.drop('contig_us_bounds', axis = 1, inplace = True) # If this column
# were left in, we would end up with the following error message when trying
# to create a map:
# TypeError: Object of type Polygon is not JSON serializable
gdf_roads

,LINEARID,FULLNAME,RTTYP,MTFCC,geometry,within_contig_us
0,1106073066667,W I- 20,I,S1100,"LINESTRING (-98.10768 32.61152, -98.07695 32.6...",True
1,110766972487,W I- 20,I,S1100,"LINESTRING (-102.76769 31.63855, -102.80296 31...",True
2,1104259293529,W I- 20,I,S1100,"LINESTRING (-103.3894 31.42685, -103.35124 31....",True
3,110206001621,W I- 20,I,S1100,"LINESTRING (-98.97076 32.37467, -99.01205 32.3...",True
4,110451633229,I- 785,I,S1100,"LINESTRING (-79.68662 36.0495, -79.6844 36.056...",True
...,...,...,...,...,...,...
5603,11012815134136,I- 10 (Hov),I,S1100,"LINESTRING (-95.45164 29.7794, -95.45015 29.77...",True
5604,11013551329905,I- 25 (Express Lanes),I,S1100,"LINESTRING (-105.00229 39.76354, -104.99257 39...",True
5605,11013551330519,I- 25 (Express Lanes),I,S1100,"LINESTRING (-105.00545 39.76025, -105.00229 39...",True
5606,11013473368094,I- 25 (Express Lanes),I,S1100,"LINESTRING (-105.0057 39.76043, -105.00229 39....",True


## Importing US state boundaries:

In [6]:
gdf_states = gpd.read_file(path_to_shapefiles+'tl_2021_us_state/tl_2021_us_state.shp')
gdf_states['geometry'] = gdf_states['geometry'].simplify(tolerance=0.001).copy()
# I found that non-US states other than DC had a 'DIVISION' value of 0. I used that fact
# below to filter gdf_states to include only the 50 states plus DC:
# gdf_states[(gdf_states['STUSPS'].isin(['AK', 'HI']) == False) & (gdf_states["DIVISION"] != '0')]
gdf_states.query("DIVISION != '0' & STUSPS not in ['AK', 'HI']", inplace = True)
gdf_states.head(5)

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,62266298634,489204185,+38.6472854,-080.6183274,"POLYGON ((-80.84632 37.4234, -80.85946 37.4294..."
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,138961722096,45972570361,+28.3989775,-082.5143005,"MULTIPOLYGON (((-83.10874 24.62949, -83.10352 ..."
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,143778561906,6216493488,+40.1028754,-089.1526108,"POLYGON ((-89.17208 37.06831, -89.17786 37.057..."
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,206232627084,18949394733,+46.3159573,-094.1996043,"POLYGON ((-92.73547 45.30157, -92.75116 45.292..."
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,25151992308,6979074857,+38.9466584,-076.6744939,"POLYGON ((-75.78872 39.65076, -75.69367 38.460..."


In [7]:
gdf_counties = gpd.read_file(
    path_to_shapefiles
    +'tl_2021_us_county/tl_2021_us_county.shp')
gdf_counties.head(5)

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477645345,10690204,+41.9158651,-096.7885168,"POLYGON ((-96.55516 41.91587, -96.55515 41.914..."
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680976231,61568965,+46.2946377,-123.4244583,"POLYGON ((-123.49077 46.38358, -123.48813 46.3..."
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016818946,29090018,+34.3592729,-104.3686961,"POLYGON ((-104.38368 34.69213, -104.37658 34.6..."
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169272970,22847034,+40.7835474,-096.6886584,"POLYGON ((-96.6814 41.04566, -96.68139 41.0456..."
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645185,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.04802 40.35066, -98.04674 40.350..."


In [8]:
starting_lat = 38
starting_lon = -95
zoom_start = 6

# Making the map's background black:

Because I prefer dark-mode desktop themes, I wanted the background of my maps to be black. However, tileless Folium maps have a relatively light background color by default. Therefore, I decided to go to https://geojson.io and draw a giant rectangle around the contiguous US that I could then use as a dark background. This is a pretty 'janky' solution, but it works for the purposes of these maps!

(See [this post](https://github.com/python-visualization/branca/issues/91#issuecomment-1166392776) by aluthfian for a more sophisticated solution.

In [9]:
us_background = gpd.read_file('us_background.json')
us_background

,geometry
0,"POLYGON ((-47.00516 -6.2409, -47.00516 69.9314..."


## Creating a map of states and interstate highways

Note: Initially, I planned to tweak the size of the Chrome webdriver window so that the US map would take up an ideal proportion of the screenshot. However, I found that some Chrome image width and height settings failed to work correctly, even though certain lower *and* higher resolution settings worked just fine.

Therefore, I instead chose to use the Pillow library (imported as PIL) to crop the screenshot created by the Chrome webdriver. In order to preserve the Leaflet logo on the bottom right, I set the map's starting latitude and longitude coordinates so that the US would be centered near the bottom right of the image. That way, in order to create my centered image, I would only have to crop out the left and top areas of the image.

In [10]:
road_style_function = lambda feature: {
        "color": "orange",
        "opacity":1,
        "weight": 2,
        "fillOpacity":0
    }

state_style_function = lambda feature: {
        "color": "white",
        "opacity":1,
        "weight": 2,
        "fillOpacity":0
    }


background_style_function = lambda feature: {
    'fillColor':'000000', 'fillOpacity':1}
# Note: I shifted my regular contiguous US starting coordinates (38 and -95)
# to the north and west so that, when cropping the image, the Leaflet copyright
# notice would remain in place
m = folium.Map([43, -107], 
               zoom_start = 7, tiles = None,
              zoom_control = False) # Since we're only interested in the
# static version of this map, there's no reason to include zoom control
# (which would clutter the screenshot)
folium.GeoJson(us_background, style_function = background_style_function).add_to(m)
folium.GeoJson(gdf_states, style_function = state_style_function).add_to(m)
folium.GeoJson(gdf_roads, style_function = road_style_function).add_to(m)
m.save('maps/states_and_roads.html')
# m

In [11]:
def create_map_screenshot(html_map_folder,
    map_filename, png_map_folder, 
    delete_html_file):
    print("Generating screenshot.")
    options = webdriver.ChromeOptions()
    # Source: https://www.selenium.dev/documentation/webdriver/browsers/chrome/
    options.add_argument(f'--window-size={driver_window_width},\
{driver_window_height}') # I found that this window
    # size, along with a starting zoom of 6 within our mapping code,
    # created a relatively detailed map of the contiguous 48 US states. 
    # If you'd like to create an even more detailed map, consider setting 
    # your starting zoom to 7 and your window size to 6000,3375.
    options.add_argument('--headless') # In my experience, this addition 
    # (which prevents the Selenium-driven browser from displaying on your 
    # computer) was necessary for allowing 4K screenshots to get saved
    # as 3840x2160-pixel images. Without this line, the screenshots would 
    # get rendered with a resolution of 3814x1868 pixels.
    # Source of the above two lines:  
    # https://www.selenium.dev/documentation/webdriver/browsers/chrome/
    # and
    # https://github.com/GoogleChrome/chrome-launcher/blob/main/docs/chrome-flags-for-tools.md
    # I learned about the necessity of using headless mode *somewhere* on 
    # StackOverflow. Many answers to this question regarding generating 
    # screenshots reference it as an important step, for instance:
    # https://stackoverflow.com/questions/41721734/take-screenshot-of-full-page-with-selenium-python-with-chromedriver/57338909

    
    # Launching the Selenium driver:
    driver = webdriver.Chrome(options=options) 
    # Source: https://www.selenium.dev/documentation/webdriver/browsers/chrome/
    
    # Navigating to the map:
    # Note: I needed to precede the local path with 'file://' as 
    # noted by GitHub user lukeis here: 
    # https://github.com/seleniumhq/selenium-google-code-issue-archive/issues/3997#issuecomment-192014472
    map_path = f"file://{html_map_folder}/{map_filename}.html"
    print(map_path)
    driver.get(map_path)
    # Source: https://www.selenium.dev/documentation/
    time.sleep(1) # Helps ensure that the browser has enough 
    # time to download
    # map contents from the tile provider. This time might need to be
    # increased if a slow internet connection is in use. Conversely,
    # if no tiles are being incorporated into the map, 
    # there may not be any need to call
    # time.sleep().
    # Taking our screenshot and then saving it as a PNG image:
    driver.get_screenshot_as_file(f"{png_map_folder}/{map_filename}.png")
    # Source: 
    # https://selenium-python.readthedocs.io/api.html#selenium.webdriver.remote.webdriver.WebDriver.get_screenshot_as_file
    
    # Exiting out of the webdriver:
    driver.quit()
    # Source: https://www.selenium.dev/documentation/
    
    if (delete_html_file == True):
        os.remove(f"{html_map_folder}/{map_filename}.html")
        print("Removed HTML copy of map.")

In [12]:
uhd_width = 3840
uhd_height = 2160

scale_factor = 2

driver_window_width = uhd_width * scale_factor
driver_window_height = uhd_height * scale_factor
driver_window_width, driver_window_height

# Note: an uhd_width of 3840, an uhd_height of 2160,
# a scale factor of 3, and a folium.Map() starting_zoom of 8
# work pretty nicely together, but the resulting 11520*6480 resolution is
# probably overkill for just about any end user.

# As discussed earlier, I had originally planned to tweak these width and 
# height settings in order to produce a a cropped map, but I ended up with
# a severely truncated output. Therefore, I instead decided to 
# use PIL to complete the cropping tasks.

(7680, 4320)

In [13]:
map_folder = os.getcwd() + '/maps'

## Creating a screenshot of the map, then deleting the HTML file on which it was based:

In [14]:
create_map_screenshot(
    html_map_folder = map_folder,
    map_filename = 'states_and_roads',
    png_map_folder = map_folder,
    delete_html_file = True)

Generating screenshot.
file:///home/kjb3/kjb3docs/programming/py/kjb3_programs_2/desktop_maps/maps/states_and_roads.html
Removed HTML copy of map.


In [15]:
## Cropping this screenshot:

In [16]:
from PIL import Image

left_crop = 2000
top_crop = left_crop * 9/16 # Initializing top_crop as
# a scaled version of left_crop ensures that the final
# image will remain in a 16:9 aspect ratio.
print(left_crop, top_crop)

with Image.open("maps/states_and_roads.png") as im:

    # The crop method from the Image module takes four coordinates as input.
    # The right can also be represented as (left+width)
    # and lower can be represented as (upper+height).
    (left, upper, right, lower) = (left_crop, top_crop, 7680, 4320)

    # Here the image "im" is cropped and assigned to new variable im_crop
    im_crop = im.crop((left, upper, right, lower))

# Overwriting the original image (since there's no reason to keep it):

im_crop.save('maps/states_and_roads.png', 'PNG')

2000 1125.0
